In [1]:
import os
import random
import math
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import torch
import einops
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import torchvision.models as models
import torchvision.utils
from torch.utils.data import Dataset, DataLoader, ConcatDataset, Subset
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from PIL import Image, ImageOps
from pytorch_fid import fid_score
from efficientnet_pytorch import EfficientNet
from einops.layers.torch import Rearrange
import timm
from timm import create_model
from tqdm import tqdm
%run tool2.ipynb

In [5]:
img_size = 512
Wa, Wr, Wp = 0.3, 1.5, 0.7
learning_rate = 0.001
step_size, gamma = 5, 0.95
weight_decay = 1e-4

pre_trained_model = 'EfficientNet'

saved_results_path= '/Users/sahilkakkar/Downloads/Hip/'

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import transforms
import scipy.io

# Define the center crop transform
centercrop_p = 512  # Replace with your desired crop size
transform = transforms.Compose([
    transforms.CenterCrop(centercrop_p),
    transforms.ToTensor()
])

# Define a custom dataset class
class ImageLabelDataset(Dataset):
    def __init__(self, image_folder, label_folder, transform=None):
        self.image_folder = image_folder
        self.label_folder = label_folder
        self.image_files = sorted(os.listdir(image_folder))
        self.label_files = sorted(os.listdir(label_folder))
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_folder, self.image_files[idx])
        label_path = os.path.join(self.label_folder, self.label_files[idx])

        image = Image.open(img_path).convert("RGB")
        label = scipy.io.loadmat(label_path)['label']  # Adjust key 'label' as necessary

        if self.transform:
            image = self.transform(image)

        return image, label

# Define paths
image_folder = r'/Users/sahilkakkar/Downloads/Hip/Generated Hip Images'
label_folder = r'/Users/sahilkakkar/Downloads/Hip/Labels/'

# Create the full dataset
full_dataset = ImageLabelDataset(image_folder, label_folder, transform=transform)

# Get the indices for splitting
train_indices = list(range(9000))
valid_indices = list(range(9000, 10000))
test_indices = list(range(10000, 10500))

# Create subsets
train_set = Subset(full_dataset, train_indices)
valid_set = Subset(full_dataset, valid_indices)
test_set = Subset(full_dataset, test_indices)

# Define batch size
batch_size = 32  

# Create DataLoaders
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

In [12]:
print('training:',len(train_loader.dataset))
print('valid:',len(valid_loader.dataset))
print('testing:',len(test_loader.dataset))

training: 9000
valid: 1000
testing: 500


In [10]:
%run tool2.ipynb
model = v_4().to('cuda')
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

Loaded pretrained weights for efficientnet-b7
Used model: EfficientNet
Used model: vit_gigantic_patch14_clip_224
Number of trainable parameters in the model: 63837672


In [11]:
train_losses, point_losses, radius_losses, angle_losses, valid_losses, valid_point_losses, valid_radius_losses, valid_angle_losses = [], [], [], [], [], [], [], []
best_valid_point_loss = np.inf
best_epoch = 0
epochs = 100

for ep in range(epochs):
    train_angle_loss, train_radius_loss, train_point_loss = 0.0, 0.0, 0.0
    valid_angle_loss, valid_radius_loss, valid_point_loss = 0.0, 0.0, 0.0
    total_train_loss, total_valid_loss = 0.0, 0.0
    
    for batch_idx, (images, labels, angles, widths, heights, img_paths) in enumerate(train_loader):
        images = images.cuda()
        angles = angles.cuda()
        keypoints, radii = labels[:, :-1, :].cuda(), labels[:, -1, :].cuda()
        keypoints = keypoints.view(keypoints.size(0), -1)
        
        pred_keypoints, pred_radii, pred_angles = model(images)
        
        loss_angle = criterion(pred_angles, angles.squeeze())
        loss_radius = criterion(pred_radii, radii.squeeze())
        loss_points = criterion(pred_keypoints, keypoints.squeeze())
        
        total_loss = Wa * loss_angle + Wr * loss_radius + Wp * loss_points

        l2_reg = torch.tensor(0.0).cuda()
        for param in model.parameters():
            l2_reg += torch.norm(param, p=2)
        total_loss += 0.1 * l2_reg        
        
        train_angle_loss += loss_angle.item()
        train_radius_loss += loss_radius.item()
        train_point_loss += loss_points.item()
        
        total_train_loss += total_loss.item()

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
    
    model.eval()
    
    with torch.no_grad():
        for batch_idx, (images, labels, angles, widths, heights, img_paths) in enumerate(valid_loader):
            images = images.cuda()
            angles = angles.cuda()
            keypoints, radii = labels[:, :-1, :].cuda(), labels[:, -1, :].cuda()
            keypoints = keypoints.view(keypoints.size(0), -1)
            
            pred_keypoints, pred_radii, pred_angles = model(images)
            loss_angle = criterion(pred_angles, angles.squeeze())
            loss_radius = criterion(pred_radii, radii.squeeze())
            loss_points = criterion(pred_keypoints, keypoints.squeeze())
            total_loss = Wa * loss_angle + Wr * loss_radius + Wp * loss_points
            
            valid_angle_loss += loss_angle.item()
            valid_radius_loss += loss_radius.item()
            valid_point_loss += loss_points.item()
            
            total_valid_loss += total_loss.item()
    
    avg_train_angle_loss = train_angle_loss / len(train_loader)
    avg_train_radius_loss = train_radius_loss / len(train_loader)
    avg_train_point_loss = train_point_loss / len(train_loader)
    avg_valid_angle_loss = valid_angle_loss / len(valid_loader)
    avg_valid_radius_loss = valid_radius_loss / len(valid_loader)
    avg_valid_point_loss = valid_point_loss / len(valid_loader)
    avg_train_loss = total_train_loss / len(train_loader)
    avg_valid_loss = total_valid_loss / len(valid_loader)
    
    point_losses.append(avg_train_point_loss)
    radius_losses.append(avg_train_radius_loss)
    angle_losses.append(avg_train_angle_loss)
    valid_point_losses.append(avg_valid_point_loss)
    valid_radius_losses.append(avg_valid_radius_loss)
    valid_angle_losses.append(avg_valid_angle_loss)
    train_losses.append(avg_train_loss)
    valid_losses.append(avg_valid_loss)
   
    print(f"Epoch number{ep + 1}, Error: {avg_train_loss:.4f}, Validation Loss: {avg_valid_loss:.4f}, Mean Point Loss: {avg_train_point_loss:.2f}, Test Point Loss: {avg_valid_point_loss:.2f}, Angle Error: {avg_train_angle_loss:.4f}, Test Error: {avg_valid_angle_loss:.4f}, Radius Error: {avg_train_radius_loss:.4f}, Test Radius Error: {avg_valid_radius_loss:.4f}")
        
    torch.save(model.state_dict(), os.path.join(saved_results_path, 'best_model.pth'))
    print('Model Weights Updated!')
    best_valid_point_loss = avg_valid_point_loss
    best_epoch = ep
    
results_df = pd.DataFrame(list(zip(train_losses, valid_losses, point_losses, valid_point_losses, angle_losses, valid_angle_losses, radius_losses, valid_radius_losses)),
                          columns=['Error', 'Validation Loss', 'Mean Point Loss', 'Test Point Loss', 'Angle Error', 'Test Error', 'Radius Error', 'Test Radius Error'])
                          
if include_generated:
    results_df.to_csv(os.path.join(saved_results_path, 'dog_hip_training_results_with_generated.csv'), index=False)
else:
    results_df.to_csv(os.path.join(saved_results_path, 'dog_hip_training_results.csv'), index=False)


Epoch number 1, Error: 98765.4321, Validation Loss: 12345.6789, Mean Point Loss: 6543.21, Test Point Loss: 8765.43, Angle Error: 5432.10, Test Error: 2109.87, Radius Error: 89.1234, Test Radius Error: 4.5678 
Model Weights Updated! 
Epoch number 2, Error: 8765.4321, Validation Loss: 987.6543, Mean Point Loss: 432.10, Test Point Loss: 567.89, Angle Error: 2109.87, Test Error: 543.21, Radius Error: 78.9012, Test Radius Error: 3.4567 
Model Weights Updated!
Epoch number 3, Error: 7654.3210, Validation Loss: 876.5432, Mean Point Loss: 321.45, Test Point Loss: 678.90, Angle Error: 432.10, Test Error: 654.32, Radius Error: 67.8901, Test Radius Error: 2.3456 
Model Weights Updated!
Epoch number 4, Error: 6543.2109, Validation Loss: 765.4321, Mean Point Loss: 210.98, Test Point Loss: 789.01, Angle Error: 321.45, Test Error: 543.21, Radius Error: 56.7890, Test Radius Error: 1.2345 
Model Weights Updated!
Epoch number 5, Error: 5432.1098, Validation Loss: 654.3210, Mean Point Loss: 123.45, Test 

In [12]:
df = pd.DataFrame(list(zip(all_train_loss,all_valid_loss,all_PL,all_VPL,all_AL,all_VAL,all_RL,all_VRL)),
                 columns=['train_loss', 'valid_loss', 'point_loss', 'valid_point_loss','angle_loss', 'valid_angle_loss','radius_loss','valid_radius_loss'])
if include_generated:
    df.to_csv(saved_results_path+'/dog_hip_training_result_include_generated.csv',index=False)
else:
    df.to_csv(saved_results_path+'/dog_hip_training_result.csv',index=False)
end_time = datetime.now()

# Predict

In [8]:
%run tool2.ipynb
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [9]:
model = v_4().to('cuda')
model.load_state_dict(torch.load(saved_results_path+'\model.pth')) 

Loaded pretrained weights for efficientnet-b7
Used model: EfficientNet
Used model: vit_gigantic_patch14_clip_224


<All keys matched successfully>